# Preprocessing .mat for NWB

In [3]:
import numpy as np
import h5py
import os
import yaml
import pandas as pd
from datetime import datetime
import shutil
import importlib
import numpy as np
import pandas as pd
import convert_to_nwb_for_PL
import converters.Initiation_nwb
import scipy.io
import warnings


# Load data 

## .mat files

In [4]:
csv_file = "data/Subject_Session_Selection.csv"
PL = "data/mouse/General"
PLALL = "data/mouse/Recording"
PLALL_number1 = "data/mouse/Recording/PL200_D1.mat"
output_folder = "data"

In [ ]:
PL200 = [os.path.join(PL, f) for f in os.listdir(PL)][0]
PL200


In [ ]:
for file_name in sorted(os.listdir(PLALL)):
    i += 1
    file_path = os.path.join(PLALL, file_name)
    if os.path.isfile(file_path) and file_name.endswith('.mat'):
        print(f"Processing file: {file_name}")

### General

In [ ]:
from scipy.io import loadmat
General_data = loadmat(PL200)

In [ ]:
type(General_data)

In [ ]:
print(General_data.keys())


In [ ]:
print(General_data["__header__"])


In [ ]:
print(General_data["__version__"])


In [ ]:
print(General_data["__globals__"])


In [ ]:

print(General_data["LFP_Data"])



In [ ]:
# Mouse Name
print(General_data["LFP_Data"][0][0][0][0][0][0])


In [ ]:
# id session 
print(General_data["LFP_Data"][0][0][4])
unique_values, unique_indices = np.unique(General_data["LFP_Data"][0][0][4], return_index=True)
print("Unique values:", unique_values)
print("Indices:", unique_indices)



In [ ]:
i=0
# Start date (dd.mm.yy)
print(General_data["LFP_Data"][0][0][5][unique_indices[1]])

# Start time (hhmmss)
## hh
hh = str(General_data["LFP_Data"][0][0][5][unique_indices[i]][3][0][0])
## mm
if len(str(General_data["LFP_Data"][0][0][5][unique_indices[i]][4][0][0])) == 1:
    mm= "0" + str(General_data["LFP_Data"][0][0][5][unique_indices[i]][4][0][0])
else:
    mm = str(General_data["LFP_Data"][0][0][5][unique_indices[i]][4][0][0])
## ss
ss = str(General_data["LFP_Data"][0][0][5][unique_indices[i]][5][0][0])

start_time = hh + mm +  ss[:2]
print("Start time:", start_time)


#Behavior Type
print(str(General_data["LFP_Data"][0][0][6][unique_indices[i]][0][0]))


In [ ]:
# Strain
print(General_data["LFP_Data"][0][0][1][0][0][0])

In [ ]:
# sex 
print(General_data["LFP_Data"][0][0][2][0][0][0])

In [ ]:
# birth date 
print(str(General_data["LFP_Data"][0][0][3][0][0][0][0]))

In [ ]:
# trial onset 
rien = unique_indices[1]
trial = np.asarray(General_data["LFP_Data"][0][0][8]).flatten()
trialonset_avant= trial[0:rien]

In [ ]:

print(General_data["LFP_Data_description"])

### ALL (number1)

In [ ]:
from scipy.io import loadmat
pli = loadmat(PLALL_number1)

In [ ]:
pli.keys()

In [ ]:
# Stim_times
stim_onset= np.asarray(pli["Stim_times"][0][0][1][0])/1000
# Catch_times
catch_onset = np.asarray(pli["Catch_times"][0][0][0][0])/1000
# trial_onset
all_onsets = np.concatenate([stim_onset, catch_onset])
all_onsets_sorted = np.sort(all_onsets)
print(all_onsets_sorted)
len(all_onsets_sorted)



In [ ]:
Valve_times= np.asarray(pli["Valve_times"][0][0][1][0])/1000
Valve_times

In [ ]:
# Compare trialonset_avant and all_onsets_sorted
trialonset_avant_sorted = np.sort(trialonset_avant)
are_equal = np.array_equal(trialonset_avant_sorted, all_onsets_sorted)
print("Are trialonset_avant and all_onsets_sorted equal?", are_equal)

# Show differences if not equal
if not are_equal:
    diff_in_avant = np.setdiff1d(trialonset_avant_sorted, all_onsets_sorted)
    diff_in_all = np.setdiff1d(all_onsets_sorted, trialonset_avant_sorted)
    print("In trialonset_avant but not in all_onsets_sorted:", diff_in_avant)
    print("In all_onsets_sorted but not in trialonset_avant:", diff_in_all)

# CSV file


In [6]:
import pandas as pd 
csv_data = pd.read_csv("data/Subject_Session_Selection.csv", sep=";")
csv_data.columns = csv_data.columns.str.strip() 
csv_data.columns

Index(['Mouse Name', 'User (user_userName)', 'Ear tag',
       'Start date (dd.mm.yy)', 'End date', 'Sex_bin', 'strain', 'mutations',
       'Birth date', 'licence', 'DG', 'ExpEnd', 'Created on', 'Session',
       'Session Date (yyymmdd)', 'Start Time (hhmmss)', 'Behavior Type',
       'Session Type', 'Mouse Age (d)', 'Weight of Reference',
       'Weight Session', 'Trial_onset', 'stim_onset', 'catch_onset',
       'Responses_times', 'EMG', 'PtA', 'dCA1', 'mPFC', 'wM1', 'wS1', 'wS2',
       'antM1'],
      dtype='object')

In [61]:
csv_row = (dataframe_csv_file.iloc[9])
csv_row

Mouse Name                                                            PL208
User (user_userName)                                                     PL
Ear tag                                                             Unknown
Start date (dd.mm.yy)                                            10.12.2014
End date                                                         10.12.2014
Sex_bin                                                                   m
strain                                                           C57BL/6JRj
mutations                                                                  
Birth date                                                          Unknown
licence                                                             Unknown
DG                                                                         
ExpEnd                                                                     
Created on                                                          Unknown
Session     

# TOTAL CSV

In [10]:
importlib.reload(converters.Initiation_nwb)
dataframe_csv_file = converters.Initiation_nwb.files_to_csv(PL=PL, PLALL=PLALL, csv_file=csv_file)
print(dataframe_csv_file.columns)
dataframe_csv_file.head(30)

Number of sessions: 8
Number of files in PLALL: 9
Processing session file: PL200_D1.mat
Processing session file: PL200_D2.mat
Processing session file: PL200_D3.mat
Processing session file: PL200_D4.mat
Processing session file: PL200_D5.mat
Processing session file: PL200_D6.mat
Processing session file: PL200_D7.mat
Processing session file: PL200_D8.mat
Index(['Mouse Name', 'User (user_userName)', 'Ear tag',
       'Start date (dd.mm.yy)', 'End date', 'Sex_bin', 'strain', 'mutations',
       'Birth date', 'licence', 'DG', 'ExpEnd', 'Created on', 'Session',
       'Session Date (yyymmdd)', 'Start Time (hhmmss)', 'Behavior Type',
       'Session Type', 'Mouse Age (d)', 'Weight of Reference',
       'Weight Session', 'Trial_onset', 'stim_onset', 'catch_onset',
       'Responses_times', 'EMG', 'PtA', 'dCA1', 'mPFC', 'wM1', 'wS1', 'wS2',
       'antM1', 'EEG'],
      dtype='object')


,Mouse Name,User (user_userName),Ear tag,Start date (dd.mm.yy),End date,Sex_bin,strain,mutations,Birth date,licence,...,Responses_times,EMG,PtA,dCA1,mPFC,wM1,wS1,wS2,antM1,EEG
0,PL208,PL,Unknown,09.12.2014,09.12.2014,m,C57BL/6JRj,NaN,Unknown,1628,...,18.638;31.433;43.728;75.424;78.12;112.112;118....,6.2e-06;-1.4432999999999999e-05;-1.1667e-05;-2...,NaN,-0.0002752;-0.000268466;-0.0002386659999999999...,-0.0001342;-0.000113534;-9.02e-05;-0.000127666...,NaN,NaN,NaN,-9.0134e-05;-7.466599999999999e-05;-5.81339999...,NaN
1,PL208,PL,Unknown,10.12.2014,10.12.2014,m,C57BL/6JRj,NaN,Unknown,1628,...,14.546;22.145;88.027;112.523;135.618;158.611;1...,8.933e-06;7.667e-06;-2.2e-05;-5.1e-06;2.3933e-...,NaN,0.00013680000000000002;0.0001108;9.2734e-05;8....,-7.580000000000001e-05;-9.02e-05;-8.3466e-05;-...,NaN,NaN,NaN,-0.000121534;-0.0001676;-0.000174334;-0.000143...,NaN
2,PL208,PL,Unknown,11.12.2014,11.12.2014,m,C57BL/6JRj,NaN,Unknown,1628,...,24.521;56.311;84.204;108.602;124.797;142.194;1...,-0.000151967;-0.0001101;-4.28e-05;1.8033e-05;4...,NaN,5.3266e-05;1.066e-06;-3.7066e-05;3.81340000000...,1.8665999999999997e-05;-1.7999999999999997e-05...,NaN,NaN,NaN,9.4334e-05;1.8866000000000002e-05;-2.1866e-05;...,NaN
3,PL208,PL,Unknown,12.12.2014,12.12.2014,m,C57BL/6JRj,NaN,Unknown,1628,...,8.628;32.12;53.414;75.507;100.402;131.696;151....,-7.333e-06;2.03e-05;8.367e-06;9.567e-06;-5e-07...,NaN,7.02e-05;7.6066e-05;6.72e-05;7.9334e-05;7.0934...,-9.9934e-05;-0.000164;-0.000117466;-8.82e-05;-...,NaN,NaN,NaN,-6.1266e-05;-6.786599999999999e-05;-0.00010466...,NaN
4,PL208,PL,Unknown,13.12.2014,13.12.2014,m,C57BL/6JRj,NaN,Unknown,1628,...,7.57;25.562;44.955;116.94;151.132;295.199;334....,3.6567000000000004e-05;-3.8e-06;3.0233e-05;1.1...,NaN,-0.0001292;-5.0066e-05;-1.22e-05;-7.0266e-05;-...,-7.246600000000001e-05;4.466e-06;2.2666e-05;-4...,NaN,NaN,NaN,-6.2734e-05;3.9134e-05;7.0466e-05;1.2534e-05;4...,NaN
5,PL208,PL,Unknown,14.12.2014,14.12.2014,m,C57BL/6JRj,NaN,Unknown,1628,...,5.476;1986.385;2000.579;2018.672;2058.565;2098...,-1.1067000000000001e-05;2.433e-06;-2.266699999...,NaN,-6.6066e-05;-8.12e-05;-0.00010273400000000001;...,-0.000190934;-0.0002228;-0.0002072;-0.00016486...,NaN,NaN,NaN,-4.8666e-05;-0.00014333399999999998;-0.0001729...,NaN
6,PL208,PL,Unknown,15.12.2014,15.12.2014,m,C57BL/6JRj,NaN,Unknown,1628,...,13.481;242.788;296.177;325.169;377.856;448.641...,5.3067000000000005e-05;-7.467e-06;-0.0001653;-...,NaN,-0.00013000000000000002;-0.0002542;-0.0002162;...,-0.000155866;-0.00011899999999999999;-7.76e-05...,NaN,NaN,NaN,-0.0001242;-5.5734e-05;5.4334e-05;5.5266e-05;-...,NaN
7,PL208,PL,Unknown,16.12.2014,16.12.2014,m,C57BL/6JRj,NaN,Unknown,1628,...,10.68;80.359;117.752;306.907;341.299;365.393;3...,2.8133e-05;3.4633e-05;3.2867e-05;7.1267e-05;2....,NaN,3.36e-05;2.8734e-05;-1.1599999999999999e-05;-1...,7.9466e-05;5.4134e-05;3.7199999999999996e-05;3...,NaN,NaN,NaN,7.3934e-05;5.199999999999999e-06;6.934e-06;6.2...,NaN
8,PL208,PL,Unknown,17.12.2014,17.12.2014,m,C57BL/6JRj,NaN,Unknown,1628,...,12.369;44.323;104.307;126.002;169.095;211.683;...,1.2e-06;-8.233e-06;2.1e-06;1.6329999999999999e...,NaN,-7.7266e-05;-7.92e-05;-7.42e-05;-6.8066e-05;-7...,2.9e-05;1.4200000000000001e-05;-4.934e-06;4.90...,NaN,NaN,NaN,2.4334e-05;-3.34e-07;1.46e-05;2.7934e-05;-3.06...,NaN
9,PL208,PL,Unknown,18.12.2014,18.12.2014,m,C57BL/6JRj,NaN,Unknown,1628,...,10.295;63.981;233.642;251.437;277.43;375.606;4...,3.6670000000000002e-06;2.6e-05;-2.67e-07;9.033...,NaN,3.1933999999999996e-05;2.66e-05;6.26e-05;4.28e...,-0.0001098;-0.0001126;-7.7866e-05;-5.8066e-05;...,NaN,NaN,NaN,-9.24e-05;-0.000126266;-0.000126666;-8.92e-05;...,NaN


In [ ]:
importlib.reload(converters.Initiation_nwb)
dataframe_csv_file = converters.Initiation_nwb.remove_rows_by_mouse_name(dataframe_csv_file, "PL208")
dataframe_csv_file.to_csv(csv_file, sep=';', index=False)

 # TOTAL _-_-_-_-_-_-_-_
 

In [18]:

importlib.reload(convert_to_nwb_for_PL)
nwb_path = convert_to_nwb_for_PL.convert_data_to_nwb_pl(csv_file=csv_file, output_folder="data", mouses_name = ["PL200"])

**************************************************************************
-_-_-_-_-_-_-_-_-_-_-_-_-_-_- NWB conversion _-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Converting data to NWB format for mouse: ['PL200_20140619', 'PL200_20140620', 'PL200_20140621', 'PL200_20140622', 'PL200_20140623', 'PL200_20140624', 'PL200_20140625', 'PL200_20140626']
📃 Creating configs for NWB conversion :
📑 Created NWB files :
     o 📌 Add general metadata
         - Subject metadata
         - Session metadata
         - Device metadata
         - Extracellular electrophysiology metadata
     o 📶 Add acquisition container
     o ⚙️ Add processing container
         - Behavior data
         - No ephys data for AN sessions
         - Analysis complementary information
 
🔎 Validating NWB file before saving...
     o ✅ File data/PL200_20140619_142055.nwb is valid, no errors detected and saved successfully.
No forget to delete the testing purpose
**************************************************************************


In [46]:
importlib.reload(converters.Initiation_nwb)
converters.Initiation_nwb.remove_nwb_files("data")